In [2]:
pip install phe

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Importing Libraries

In [1]:
from phe import paillier
import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

Key Generation Module

In [5]:
from sklearn.model_selection import train_test_split

class Client:
    def __init__(self, id, public_key, private_key, data, labels):
        self.id = id
        self.public_key = public_key
        self.private_key = private_key

        # Split the data into train and test sets
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(data, labels, test_size=0.3, random_state=42)

        self.model_weights = None
    
    def train_local_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01)
        clf.fit(self.X_train, self.y_train)
        self.model_weights = clf.coef_[0]
        return self.model_weights

    def encrypt_weights(self):
        return [self.public_key.encrypt(x) for x in self.model_weights]
    
    def decrypt_weights(self, encrypted_weights):
        return np.array([self.private_key.decrypt(x) for x in encrypted_weights])

Server-Side Logic

In [6]:
class Server:
    def __init__(self, public_key):
        self.public_key = public_key
        self.encrypted_models = []

    def receive_encrypted_model(self, encrypted_model):
        self.encrypted_models.append(encrypted_model)

    def aggregate_encrypted_models(self, num_clients):
        n_weights = len(self.encrypted_models[0])
        aggregated_model = []
        for i in range(n_weights):
            sum_enc = sum(client_weights[i] for client_weights in self.encrypted_models)
            avg_enc = sum_enc * (1 / num_clients)
            aggregated_model.append(avg_enc)
        return aggregated_model

Simulated Federated Training

In [7]:
def federated_training(num_clients=3, num_rounds=3):
    # Step 1: Create data
    X, y = make_classification(n_samples=300, n_features=5, n_informative=3, n_classes=2)
    split_data = np.array_split(X, num_clients)
    split_labels = np.array_split(y, num_clients)

    # Step 2: Generate key pair
    public_key, private_key = paillier.generate_paillier_keypair()

    # Step 3: Initialize clients
    clients = []
    for i in range(num_clients):
        clients.append(Client(i, public_key, private_key, split_data[i], split_labels[i]))

    # Step 4: Initialize server
    server = Server(public_key)

    for round in range(num_rounds):
        print(f"\n🔁 Round {round + 1} - Local training and encryption")

        server.encrypted_models.clear()
        for client in clients:
            local_weights = client.train_local_model()
            enc_weights = client.encrypt_weights()
            server.receive_encrypted_model(enc_weights)
            print(f"Client {client.id} encrypted and sent weights.")

        print("🔐 Server aggregating encrypted models...")
        encrypted_global_model = server.aggregate_encrypted_models(num_clients)

        for client in clients:
            decrypted_global_model = client.decrypt_weights(encrypted_global_model)
            client.model_weights = decrypted_global_model  # Update local model weights
            print(f"Client {client.id} decrypted global model.")

    return clients

# ------------------------------
# Evaluation
# ------------------------------

def evaluate_global_model(clients):
    for client in clients:
        clf = SGDClassifier()
        clf.coef_ = client.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([0])
        clf.classes_ = np.array([0, 1])  # Manually set class labels

        preds = clf.predict(client.X_test)
        acc = accuracy_score(client.y_test, preds)
        print(f"Client {client.id} Test Accuracy: {acc:.2f}")


# ------------------------------
# Run the Whole Pipeline
# ------------------------------

clients = federated_training()
evaluate_global_model(clients)


🔁 Round 1 - Local training and encryption
Client 0 encrypted and sent weights.
Client 1 encrypted and sent weights.
Client 2 encrypted and sent weights.
🔐 Server aggregating encrypted models...
Client 0 decrypted global model.
Client 1 decrypted global model.
Client 2 decrypted global model.

🔁 Round 2 - Local training and encryption
Client 0 encrypted and sent weights.
Client 1 encrypted and sent weights.
Client 2 encrypted and sent weights.
🔐 Server aggregating encrypted models...
Client 0 decrypted global model.
Client 1 decrypted global model.
Client 2 decrypted global model.

🔁 Round 3 - Local training and encryption
Client 0 encrypted and sent weights.
Client 1 encrypted and sent weights.
Client 2 encrypted and sent weights.
🔐 Server aggregating encrypted models...
Client 0 decrypted global model.
Client 1 decrypted global model.
Client 2 decrypted global model.
Client 0 Test Accuracy: 0.87
Client 1 Test Accuracy: 0.80
Client 2 Test Accuracy: 0.77
